In [7]:
import queue

import dataclasses
import typing

class CallbackQueue:
    @dataclasses.dataclass
    class Task:
        func: typing.Callable
        result: typing.Any = None

    def __init__(self, maxsize=1):
        self.base = queue.Queue(maxsize=maxsize)

    def __call__(self, *args, **kwargs):
        task: self.Task = self.base.get()
        task.result = task.func(*args, **kwargs)
        self.base.task_done()

    def call(self, func: callable):
        task = self.Task(func=func)
        self.base.put(task)
        self.base.join()
        return task.result

class CloseableCallbackQueue(CallbackQueue):
    class Closed(Exception):
        pass

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.closed = False

    def __call__(self, *args, **kwargs):
        if self.closed:
            raise self.Closed()
        return super().__call__(*args, **kwargs)
    
    def call(self, func: callable):
        if self.closed:
            raise self.Closed()
        return super().call(func)
    
    def close(self):
        self.closed = True
        # NOTE ref https://stackoverflow.com/a/18873213
        while not self.base.empty():
            try: self.base.get(block=False)
            except queue.Empty: continue
            self.base.task_done()

In [8]:
def _TODO_act():
    while True:
        # TODO set ACTION: do stuff
        # TODO next step
        yield
        # TODO get RESULT: obs, reward, done, info

In [9]:
import energyplus.ooep
import energyplus.ooep.ems


# TODO
import queue
_queue = queue.Queue(maxsize=1)


def _f_callback():
    global env
    if env.warming_up:
        return

    # TODO
    # next(state)

    global _queue
    print('cb: WAITING', (_queue).qsize())

    #asyncio.ensure_future(_queue.put('test item'), loop=loop)
    func = _queue.get()
    func()
    _queue.task_done()

    #assert (_queue).qsize() == 0
    print('cb: WAITING DONE', (_queue).qsize())
    #_queue.join()


env = energyplus.ooep.ems.Environment().__enter__()

cb = CloseableCallbackQueue()
env.event_listener.subscribe(
    #dict(event_name='begin_zone_timestep_after_init_heat_balance'),
    #dict(event_name='after_component_input'),
    dict(event_name='begin_new_environment'),
    #_f_callback
    cb,
)


from energyplus.dataset.basic import dataset as epds

In [10]:
import functools
import threading

def _TODO_env():
    global _queue
    env(
        # TODO
        '--design-day',
        #'--annual',
        '--output-directory', 'build/demo-eplus',
        '--weather', f'{epds.weathers}/USA_FL_Tampa.Intl.AP.722110_TMY3.epw',
        f'{epds.models}/ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf',
        verbose=True,
    )
    print('END', _queue.qsize())

    global cb
    cb.close()


thr = threading.Thread(target=_TODO_env)

thr.start()



EnergyPlus Starting
EnergyPlus, Version 23.2.0-7636e6b3e9, YMD=2023.12.26 19:44


Initializing Response Factors
Calculating CTFs for "INTERIORFURNISHINGS"
Calculating CTFs for "DROPCEILING"
Calculating CTFs for "INT_WALL"
Calculating CTFs for "EXT_SLAB_8IN_WITH_CARPET"
Calculating CTFs for "INT_SLAB_FLOOR"
Calculating CTFs for "NONRES_ROOF"
Calculating CTFs for "NONRES_EXT_WALL"
Calculating CTFs for "BASEMENT_WALL_EAST_CFACTOR"
Calculating CTFs for "BASEMENT_WALL_SOUTH_CFACTOR"
Calculating CTFs for "BASEMENT_WALL_NORTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_NORTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_SOUTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_WEST_CFACTOR"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing S

In [14]:
cb.call(lambda: ('new env'))

Closed: 

In [24]:
thr.is_alive()

False

In [ ]:
# TODO
# state_update
# on_state_update

In [136]:
def _TODO_():
    while True:
        print('prev obs', flush=True)
        yield # actions
        print('obs', flush=True)


todo = _TODO_()
import time
while True:
    print('sim', flush=True)
    time.sleep(10)
    next(todo)


# TODO
# def step(action): todo.send(None); obs = await todo.recv();

sim


KeyboardInterrupt: 

In [ ]:
# action -> resp

In [142]:
todo.send(None)

obs
prev obs


In [125]:
_queue.get()

Initializing Response Factors
Calculating CTFs for "INTERIORFURNISHINGS"
Calculating CTFs for "DROPCEILING"
Calculating CTFs for "INT_WALL"
Calculating CTFs for "EXT_SLAB_8IN_WITH_CARPET"
Calculating CTFs for "INT_SLAB_FLOOR"
Calculating CTFs for "NONRES_ROOF"
Calculating CTFs for "NONRES_EXT_WALL"
Calculating CTFs for "BASEMENT_WALL_EAST_CFACTOR"
Calculating CTFs for "BASEMENT_WALL_SOUTH_CFACTOR"
Calculating CTFs for "BASEMENT_WALL_NORTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_NORTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_SOUTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_WEST_CFACTOR"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing S

In [126]:
while thr.is_alive():
    #import time
    #time.sleep(1)
    _queue.put(lambda: ...)
    _queue.join()
    print('step qsize get', _queue.qsize(), thr.is_alive())
    if not thr.is_alive():
        break

#print(_queue.qsize())
thr.join()

step qsize get 0 True
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb

Warming up
Warming up
Warming up
Warming up
Warming up
Performing Zone Sizing Simulation
...for Sizing Period: #2 DENVER-AURORA-BUCKLEY.AFB_CO_USA ANN CLG .4% CONDNS DB=>MWB
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
cb: WAITING 0
cb: WAITING DONE 1
step qsize get 0 True
c

EnergyPlus Completed Successfully.


KeyboardInterrupt: 

In [ ]:
# TODO callback
# TODO func step(action: callable): wait for `action` to be performed; ret observations;
# TODO get; get obs; task_done